# **Extracting TESS light curves with `lightkurve`**
## **AAVSO Data-Mining Workshop, Nov. 1, 2020**
---------------------------

### Updated April 7, 2021: this version is compatible with the `mkpy3` finder-chart software as well as version 2.0 of `lightkurve`.

---------------------------

### Summary

This notebook uses the interactive light-curve inspection tool from lightkurve to extract light curves from TESS data. I have structure it so that the user will not need to change any code in order to make it work; you do not need to know Python to execute it. I've divided this notebook into four blocks of code, which I've labeled as Cells A, B, C, and D. Simply run the three cells in order, following the instructions at the top of each cell. Cell D is optional; use it only if you want to create a datafile that can be loaded into Peranso.

The major limitation with this notebook is its one-size-fits-all approach to background subtraction. This technique (applied in Cell C) works well enough for variable stars with large amplitudes, but for high-precision work, such as detecting exoplanet transits, a more sophisticated background-subtraction algorithm would be very helpful. This can be achieved, for example, with `lightkurve` or `eleanor`, but these approaches require some familiarity with Python.

--------------
### Sample targets for the workshop

> For everyone:
> * XZ Cyg (a bright RR Lyrae star)
> * Thuban (Alpha Draconis; discovered to be an eclipsing binary with TESS)
> * ST Tri (FFI; multiple variable stars)
> * LINEAR 20121650 (overcontact binary; FFI; good example of background artifacts)
> * V808 Aur (FFI; good example of a spurious brightening caused by blending with a minor planet)

> Eclipsing binaries
> * AZ Cir (FFI; unknown period)
> * BB Cha (FFI; unknown period)
> * BP Hyi (FFI; unknown period)
> * V457 Lac (FFI; has an ambiguous period --- either 6.157 d or half that value)

> Short-period pulsators:
> * SW Boo (short Blazko period; fundamental period is changing)
> * VX Hya (multiple periods)
> * AC And (multiple periods)
> * RR Gem (fundamental period is changing)

---------------
# **_Cell A: Search for observations of a target_**
---------------
> **Purpose of Cell A:** Cell A will search for TESS observations (both fast-cadence and FFI) of a specified target.
>
> **Instructions for Cell A:**
> 1. Run this cell, and when prompted, enter the name of the target or the J2000 coordinates. As an example, acceptable coordinate formats for RA = 12h 34m 56s, Dec = +12d 34m 56s are: 
>* `12h34m56s+12d34m56s`
>* `12h 34m 56s +12d 34m 56s`
>* `12 34 56 +12 34 56`
>* `12 34 56, +12 34 56`
> 2. A table summarizing available observations will appear underneath Cell A. You will need this table for Cell B.
>
> **Known failure modes:**
>  * This cell requires an Internet connection; a bad connection can result in a `ConnectionResetError`. If this happens, try re-running the cell.
>  * If the object name is not known to the Simbad service, you'll get a lengthy `NameResolveError`. If that happens, go to the [aavso.org/vsx], query the object, and retrieve its coordinates. Use them instead of the name here.

In [ ]:
#################################################################
# NO USER INPUT REQUIRED IN THIS CELL'S CODE. Just run it as-is.
#################################################################
from lightkurve.search import search_tesscut, search_targetpixelfile
from astropy.coordinates import SkyCoord
import astropy.units as U
from copy import deepcopy

# Ask the user to input the name of the target.
target_name = input("Enter the name or RA/Dec (J2000) of the object:  ")

# First, we'll see if the user entered a particular format
# for equatorial coordinates. If not, the ensuing except statement
# is flexible and can accomodate other coordinate formats, as well
# as an object's common name.
try:
    position = SkyCoord(target_name, unit=(U.hourangle, U.deg))

# If astropy didn't recognize the input as a set of equatorial
# coordinates, it raises a Value Error. In this situation, 
# we'll query Simbad to see whether it's the recognized 
# name of an object. The .from_name() method also works with
# some coordinate formats.
except ValueError:
    print("\nQuerying Simbad. This may take a moment.")
    
    position = SkyCoord.from_name(target_name) 
    print("Coordinates successfully retrieved.")

# Now query MAST to generate a table indicating the available Kepler/TESS
# observations for that target.
print("Searching for fast-cadence observations of %s.\nThis can take several minutes.\n" 
      % target_name)

# First, we'll check for 2-min-cadence data.
results = search_targetpixelfile(position,
                                 radius = 120, # search radius = 120 arcsec
                                 mission = 'TESS',
                                 cadence = 'short')


# It's helpful to keep track of whether we're using FFI data. By default, 
# we are not.
using_FFI_data = False

# If there's are fast-cadence datasets, print a summary of them and skip
# the FFI query.
if len(results) > 0:
    print("Fast cadence observations are available. Forgoing FFI query.\n")

    # Print a table with important information.
    print(25*'-', "\n\nThe following fast-cadence datasets are available for download.\n")
    print(results)
        

# As a fallback, we'll check for FFI data.
else:
    print("No 2-min-cadence data available. Now checking FFI data with TESSCut.\n")
    FFIresults = search_tesscut(position)
    using_FFI_data = True
    results = FFIresults

if using_FFI_data & (len(results) > 0):
    # Print a table with important information.
    print(25*'-', "\n\nThe following FFI datasets are available for download.\n")
    # Deal with the version issue of lightkurve noted above. Different versions use different
    # names for the columns.
    print(results)
    
elif len(results) == 0: 
    print("\n\n*** No TESS observations are available for the designated target. ***\n"
         "Consider checking the TESS Web Visibility Tool to confirm whether "
         "your target has been, or will be, observed by TESS.")

> --------------
> ### **Which information do I need from this table?**
> --------------
> Choose the sector of observations that you want, and note the value of the **`#`** column. **In Cell B, you'll use this number to download that dataset.**

__________________________
# **_Cell B: Download a dataset_**
___________________________

>**Purpose of Cell B:** After you run Cell A, there will be a table underneath it that indicates how many datasets are available for download, with each row summarizing a different dataset. However, Cell A does not download any of these datasets; that's what we do in Cell B.
>
> **Instructions for Cell B:**:
>1. Consult the table from Cell A and decide which observation you wish to download. Note the corresponding value from the table's `#` column.
>2. Run Cell B, and when prompted, enter the value of `#` that corresponds with the desired dataset.
>3. Some TESS images have known data-quality issues, and you have the option of excluding them by specifying a "quality bitmask." Use the following table (based on the [lightkurve documentation](https://docs.lightkurve.org/api/lightkurve.targetpixelfile.TessTargetPixelFile.html) ) to decide which of the four options (**none, default, hard, hardest**) to use.
>>
>> | Quality bitmask | Description |
>> |---|---|
>> | **none** | all data, regardless of quality, will be used
>> | **default** | excludes data with severe quality issues
>> | **hard** | excludes data with less serious quality issue
>> | **hardest** | excludes all data with known quality issues
>>
> The notebook will ask you to choose one of the these options; **hard** is a good choice if you're not sure. (Advanced  users can specify an integer threshold in the code.)
>4. If you are using FFI data, you will also be prompted for the desired dimensions of the cutout in pixels.

In [ ]:
########################################################################
### NO USER INPUT REQUIRED IN THIS CELL'S CODE. Just run it as-is.   ###
########################################################################

# Ask the user for the row number from the table.
row_number = int(input("Enter the value of # for the desired dataset:  "))

# Print an error message for invalid input.happens
assert row_number in results.table['#'], "Invalid value of #. Please re-run the cell and try again."

# Ask for the quality bitmask and de-capitalize the input.
quality = input("\nSpecify the quality requirement. Options are none, default,\n"\
                "hard, or hardest.    ").lower()

# Check that one of those four quality bitmasks was specified.
assert quality in ['none', 'default', 'hard', 'hardest'], "Invalid quality bitmask. Choose from one of the four listed options."


# For FFI data only, we must specify the dimensions of the image to return.
if using_FFI_data:
    print("\nWith FFI data, you must specify the desired dimensions of the image.\n")
    cutout_X = int(input("How many pixels in the X direction? "))
    cutout_Y = int(input("How many pixels in the Y direction? "))
    print("\nDownloading dataset. This might take several seconds.")
    
    tpf = results[row_number].download(cutout_size = (cutout_Y, cutout_X), 
                                       quality_bitmask = quality)
    
    # We create a copy of the original because we will need to subtract
    # the background in Cell C. 
    tpf_raw = deepcopy(tpf)
    
else:
    print("\nDownloading dataset. This might take several seconds.")
    tpf = results[row_number].download(quality_bitmask = quality)

print("\n\nDataset has been downloaded and saved in:\n %s" % tpf.path )

--------------------------
# **_Cell C: Extract and save a light curve_**
---------------------------
> **Purpose of Cell C:** This cell creates an interactive widget that lets the user extract a light curve from the target-pixel file. It can then save the light curve in .fits format. If you are working with FFI data, this cell will also apply a background subtraction.
>
> **Directions for Cell C:** 
>1. You will be prompted to specify the filename under which to save the light curve that you produce. (The default filenames are too long to be useful.) You will also be given the option of generating a finder chart.
>2. An interactive widget will appear underneath the cell; directions will appear underneath it. Use this widget to select which pixels to extract to create the light curve of your target. If you are using FFI data, ignore the warning that there are `No pixels in aperture_mask`.
>3. Save your light curve when you are satisfied. You're done!
>
> **Finder charts**
>
> The optional finder-chart software, called `mkpy3`, was created by Ken Mighell at NASA Ames. It needs to be installed separately along with `reproject`, a library that allows astronomical imagery to be resampled. If you want finder charts, run the following two commands in the Anaconda prompt or in the terminal (for Mac/Linux users).
>1. `conda install reproject -c conda-forge`
>2. `python -m pip install mkpy3`
>
> **Troubleshooting a common error**
>
> Check your current URL. By default, the URL should begin with `localhost:8888`, but if you are running multiple notebook servers, the four-digit number might differ from the default of 8888. The interactive tool created by this cell is sensitive to this number, and by default, it assumes that it is 8888. If the four-digit number is not 8888, you'll get an error message similar to:
>>`ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'http://localhost:8888'`
>
> Luckily, there's a quick solution to this problem.
> 1. Note the four-digit number, and go to the very last line of code in Cell C.
> 2. Change `notebook_url='localhost:8888` so that the four-digit number matches the number in the URL.
> 3. Re-run the cell.

In [ ]:
#################################################################
# NO USER INPUT REQUIRED IN THIS CELL'S CODE. Just run it as-is.
#################################################################

# lightkurve's default filenames are often too long to be useful,
# so we will let the user specify the filename under which to save
# the light curve.
output_filename = input("When you save your light curve, it will be exported " \
                        "as a .fits file.\nWhat would you like the filename "\
                        "to be? Omit the .fits extension.\n")

# See if mkpy3 is installed. If so, ask the user whether they want a finder chart.
try:
    import mkpy3
    user_wants_a_finder = input("Do you want to generate a finder chart? [yes/no]" ).lower()
# If mkpy3 isn't installed, just keep going.
except ImportError:
    user_wants_a_finder = 'n'

# For FFI data ONLY: We need to subtract the background. Unlike the fast-cadence data,
# This isn't done by default for FFI data, so it's a problem left to the end-user. 
#
# Here, we will use a one-size-fits-all solution: we will assume that the background
# level is uniform in each frame and estimate it as the average brightness of the
# faintest 10% of pixels in a given frame. This doesn't work as well if the background
# has a brightness gradient. While a more rigorous treatment of the background would 
# be desirable, but it would also require some knowledge of Python.

if using_FFI_data:

    # We need numpy to subtract the background.
    import numpy as np

    # create an empty list for saving the background-subtracted images
    flux_sub = []

    # iterate across the images, measure the background in each image,
    # and subtract it. 
    for image in tpf_raw.hdu[1].data['FLUX']:
        # Here, we estimate the background as the average brightness
        # of the faintest 10% of pixels in each frame.
        background = image[image <= np.percentile(image, 10)].mean()
        flux_sub.append(image - background)
    # We now update the flux of the TPF.
    tpf.hdu[1].data['FLUX']= np.array(flux_sub)

# If the user asked for a finder chart, create one using 
# Ken Mighell's mkpy3.
if user_wants_a_finder in ['yes', 'y']:
     mkpy3.mkpy3_tpf_overlay_v6(tpf = tpf, verbose = False, 
                                print_gaia_dr2 = False, width_height_arcmin=6,
                                gaia_dr2_kwargs_str = "None", percentile=99.5,
                                title = 'Finder for %s' % target_name, 
                                rotationAngle_deg = "tpf")

# Create the interactive widget. The optional argument will cause the output .fits
# file to have the filename specified by the user.
tpf.interact(exported_filename='%s.fits' % output_filename, notebook_url='localhost:8888')

**A quick overview of the interactive widget created by `tpf.interact()`:**

* The image on the right shows the pixels from one frame in the target-pixel file. The highlighted pixels (called the "aperture") indicate which pixels are being measured to create the light curve in the right-hand panel. This is done by summing their intensities.
* The pixel colors of the TPF are used to represent the observed intensity at a given pixel.
* To select a custom aperture, shift-click on pixels.
* The slider underneath the light curve lets you view different images in the TPF. This is useful if you want to inspect the original data for an artifact that might have produced a suspicious feature in the light curve.
* The slider underneath the image adjusts the display contrast for the TPF frame; it does not alter the underlying data.
* When you're satisfied with your exraction, click the green 'Save Lightcurve' button to save your custom extraction to a .fits file.

---------------------------
# **I've saved my light curve. What's next?**
---------------------------
> You may analyze the light curve with the software of your choice. At the AAVSO workshop, we will use the AAVSO's VStar software, but if you're comfortable with Python, you can analyze it with `astropy`, `lightkurve`, etc. In VStar, you will need to use the lightkurve plugin.

# **Supplemental resources**

> This notebook presumes no prior knowledge of Python and therefore goes out of its way to prevent you from needing to interact directly with the code. However, if you already have a background in Python, you can carry out a more sophisticated and rigorous analysis than is possible here. Consider the following resources:
> * Exoplanet detection tutorial: https://docs.lightkurve.org/tutorials/02-recover-a-planet.html
> * Period analysis in lightkurve of XZ Cyg: https://github.com/KenMighell/mkpy3/blob/master/mkpy3/docs/source/tutorials/nb_xz_cyg_tess_v3.ipynb
> * Full list of lightkurve tutorials (of varying complexity): https://docs.lightkurve.org/tutorials/index.html
> * eleanor (an alternative Python package for FFI data): https://adina.feinste.in/eleanor/
# **Acknowledgments**

The following individuals have contributed to this notebook:
* Gordon Myers, Ken Hudson, John Giudice (members of the AAVSO-TESS pilot project)
* Ken Mighell (NASA Ames; developed the finder-chart package `mkpy3`)
* David Benn and Maksym Pyatnytskyy (wrote a special VStar plugin for lightkurve-produced files)